In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.initializers import glorot_uniform, RandomNormal
import random, os, pickle, copy
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
savedir = os.path.join('./save/find_smallest_network_cifar5')

Using TensorFlow backend.


In [2]:
def map_to_range(arr):
    narr = np.zeros_like(arr, dtype=np.int64)
    vals = np.unique(arr)
    for i, v in enumerate(vals):
        mask = (arr == v)
        narr[mask] = i
    return narr

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
input_shape = (32, 32, 3)
num_classes = 5

y_test = y_test.astype(np.int64).squeeze()
y_train = y_train.astype(np.int64).squeeze()

# labels found to give roughly equal classification performance
set_1_test_mask = np.isin(y_test, np.array([0, 3, 4, 6, 7])).squeeze()
set_1_train_mask = np.isin(y_train, np.array([0, 3, 4, 6, 7])).squeeze()

set_2_test_mask = np.isin(y_test, np.array([1, 2, 5, 8, 9])).squeeze()
set_2_train_mask = np.isin(y_train, np.array([1, 2, 5, 8, 9])).squeeze()

set_1_train_labels = y_train[set_1_train_mask]
x_train_1 = x_train[set_1_train_mask]
set_1_test_labels = y_test[set_1_test_mask]
x_test_1 = x_test[set_1_test_mask]
set_2_train_labels = y_train[set_2_train_mask]
x_train_2 = x_train[set_2_train_mask]
set_2_test_labels = y_test[set_2_test_mask]
x_test_2 = x_test[set_2_test_mask]

y_train_1 = keras.utils.to_categorical(map_to_range(set_1_train_labels), num_classes)
y_train_2 = keras.utils.to_categorical(map_to_range(set_2_train_labels), num_classes)
y_test_1 = keras.utils.to_categorical(map_to_range(set_1_test_labels), num_classes)
y_test_2 = keras.utils.to_categorical(map_to_range(set_2_test_labels), num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
datagen1 = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen1.fit(x_train_1)

datagen2= ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen2.fit(x_train_2)

In [5]:
def set_up_model(lr=1e-3, nfilters=[2, 4, 8], ndense=128):
    keras.backend.clear_session()
    inputs = Input(shape=input_shape)
    x = Conv2D(nfilters[0], kernel_size=(7,7), strides=(1, 1), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Conv2D(nfilters[1], kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Conv2D(nfilters[2], kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Flatten()(x)
    x = Dense(ndense, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(5, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adam(lr=lr),
                  metrics=['accuracy'])
    return model

In [15]:
epochs = 1
batch_size = 128

n_dense = [32, 64, 128]
constant_filters = [2, 4, 8]
exp_filters = [1, 2, 3]
for d in n_dense:
    for f in constant_filters:
        model = set_up_model(nfilters=[f, f, f], ndense=d)
        history = model.fit_generator(datagen1.flow(x_train_1, y_train_1, batch_size=batch_size),
                                      epochs=epochs,
                                      verbose=2,
                                      validation_data=(x_test_1, y_test_1),
                                      )
        with open(os.path.join(savedir, 'history_%d_dense_%d_filters.pkl'%(d, f)), 'wb') as f:
            pickle.dump(history.history, f)
    for f in exp_filters:
        model = set_up_model(nfilters=[2**(f), 2**(f+1), 2**(f+2)], ndense=d)
        history = model.fit_generator(datagen1.flow(x_train_1, y_train_1, batch_size=batch_size),
                                      epochs=epochs,
                                      verbose=2,
                                      validation_data=(x_test_1, y_test_1),
                                      )
        with open(os.path.join(savedir, 'history_%d_dense_exp_%d_filters.pkl'%(d, f)), 'wb') as f:
            pickle.dump(history.history, f)

Epoch 1/1


KeyboardInterrupt: 